In [ ]:
%load_ext autoreload
%autoreload 2

import pickle
import yaml
import copy
import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

import scipy.interpolate

import cartopy

from netCDF4 import Dataset

import wandb

from dataset import PINNDataset
from model import PINN
from visualization import GeneratorVisualizationCallback

In [ ]:
matplotlib.use('svg')
figure_params = { "font.family": 'Times', "font.size": 12, "font.serif": [], "svg.fonttype": 'none'}
matplotlib.rcParams.update(figure_params)

In [ ]:
data_dir = '/home/teisberg/data/common_data/'
nc_measure = Dataset(os.path.join(data_dir, 'antarctica_ice_velocity_450m_v2.nc'))

In [ ]:
def load_model(model_string):
    # Download save data
    api = wandb.Api()
    run = api.run(model_string)
    for file in run.files():
        if file.name.startswith('gen_model/'):
            file.download(root='downloaded_model/', replace=True)
        if file.name == 'config.yaml':
            file.download(root='downloaded_model/', replace=True)
    
    # Build model

    model_path = 'downloaded_model/gen_model'
    parameter_yaml_filename = 'downloaded_model/config.yaml'

    #
    # Parameters
    #

    with open(parameter_yaml_filename) as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
        
    for k in config:
        if isinstance(config[k], dict):
            config[k] = config[k]['value']

    config['wandb'] = False
    #print(config)

    #
    # Data Loading
    #

    with open(config['input_data_filename'], 'rb') as f:
        data = pickle.load(f)

    dataset = PINNDataset(data, batch_size=config['batch_size'],
                            mode=config['mode'], n_random=config['n_random_points'])

    #
    # Create model
    #

    model = PINN(config, dataset, gen_model_filename=model_path)
    
    return config, dataset, model

In [ ]:
model_strings = {
    'Cartesian, Symmetric': 'teisberg/igarss2021/1zguuhhe',
    'Polar, Asymmetric, Diff Smooth': 'teisberg/igarss2021/1p369fae'
}

models = {}
configs = {}
visualizers = {}
for k in model_strings:
    print(f"Loading model {k} [{model_strings[k]}]")
    config, dataset, model = load_model(model_strings[k])
    viz = GeneratorVisualizationCallback(config, dataset, model)
    models[k] = model
    configs[k] = config
    visualizers[k] = viz

In [ ]:
roi = {
    'x0': 410000,
    'x1': 560000,
    'y1': -780000,
    'y0': -930000,
    'spacing': 500
}

preds = {}
for k in models:
    print(f"Working on {k}")
    roi_x, roi_y, pred = visualizers[k].roi_prediction(roi)
    preds[k] = {pk: np.reshape(pred[pk], np.shape(roi_x)) for pk in pred}

In [ ]:
# Interpolate surface velocities
x = np.array(nc_measure.variables['x'])
y = np.array(nc_measure.variables['y'])
mask_x = (x > roi['x0']) & (x < roi['x1'])
mask_y = (y > roi['y0']) & (y < roi['y1'])
measure_X, measure_Y = np.meshgrid(x[mask_x], y[mask_y])
measure_vx = np.array(nc_measure.variables['VX'])[mask_y, :][:, mask_x]
measure_vy = np.array(nc_measure.variables['VY'])[mask_y, :][:, mask_x]

surf_vx = scipy.interpolate.griddata((measure_X.flatten(), measure_Y.flatten()), measure_vx.flatten(),
                                   (roi_x, roi_y))
surf_vy = scipy.interpolate.griddata((measure_X.flatten(), measure_Y.flatten()), measure_vy.flatten(),
                                   (roi_x, roi_y))
surf_v = np.sqrt(surf_vx**2 + surf_vy**2)

In [ ]:
# model_mapping = [
#     ('Symmetric Error Bounds', 'Baseline'),
#     ('Asymmetric Bounds + Difference Smoothing', 'Velocity Smoothing [- Only]')
# ]
model_mapping = [(k,k) for k in models]

fig, axs = plt.subplots(2, len(model_mapping)+2, figsize=(16,10), sharex=True, sharey=True)

norm = matplotlib.colors.Normalize(vmin=0, vmax=500)
diff_norm = matplotlib.colors.Normalize(vmin=-50, vmax=50)

pcm = axs[0,-2].pcolormesh(roi_x/1000,roi_y/1000,surf_v,norm=norm, rasterized=True, shading='nearest')
axs[0,-2].set_aspect('equal')
axs[0,-2].set_title('Surface Velocity')

for idx, m in enumerate(model_mapping):
    k = m[1]
    v = np.sqrt(preds[k]['vx']**2 + preds[k]['vy']**2)
    pcm = axs[0,idx].pcolormesh(roi_x/1000,roi_y/1000,v,norm=norm, rasterized=True, shading='nearest')
    axs[0,idx].set_aspect('equal')
    if idx == 0:
        clb = fig.colorbar(pcm, ax=axs[0,-1])
        clb.set_label('Predicted Depth-Averaged Velocity')
    
    axs[0,idx].set_title(m[0])
    
    pcm = axs[1,idx].pcolormesh(roi_x/1000,roi_y/1000,v - surf_v, norm=diff_norm, cmap='coolwarm', rasterized=True, shading='nearest')
    axs[1,idx].set_aspect('equal')
    if idx == 0:
        clb = fig.colorbar(pcm, ax=axs[1,-1])
        clb.set_label('Surface - Depth-Averaged Velocity')
    
    if idx == 0:
        axs[0,idx].set_ylabel('Y [km]')
        axs[1,idx].set_ylabel('Y [km]')
    axs[1,idx].set_xlabel('X [km]')
        
fig.savefig('figures/raw-vel-comparison-2.svg', format='svg', dpi=1000)